# Etude d'un circuit RC

## Import des modules python

In [3]:
import numpy as np
import scipy.integrate as itg
import matplotlib.pyplot as plt
from matplotlib import rc
rc('text', usetex=True)

## Paramètres du circuit

In [21]:
R=1e3 # en Ohms
C=1e-6 # en Farad

if R*C>0.1:
    print("Constante de temps du circuit, RC={:.2f}s".format(R*C))
elif R*C>1e-4:
    print("Constante de temps du circuit, RC={:.2f}ms".format(R*C*1e3))
elif R*C>1e-7:
    print("Constante de temps du circuit, RC={:.2f}µs".format(R*C*1e6))     
else:
    print("Constante de temps du circuit, RC={:.3f}ns".format(R*C*1e9))

Constante de temps du circuit, RC=1.00ms


## Réponse à un échelon de tension

### Définition de la tension d'excitation
La tension aux bornes du condensateur $u(t)$ est régie par l'équation différentielle suivante
$$ u(t) + \tau\frac{\text{d}u(t)}{\text{d}t}=E(t),$$ où $E(t)$ est la tension excitatrice et $\tau=RC$. Cette équation peut s'écrire comme suit $$\frac{\text{d}u(t)}{\text{d}t}=\frac{E(t)-u(t)}{\tau}.$$
Cette forme est adaptée pour une résolution numérique avec le module `scipy.integrate.odeint`. Ce module permet de résoudre une équation différentielle de la forme `dy/dt = func(y, t0, ...)` où `y` est un vecteur, `t_0` un instant initial et `func()` une fonction préalablement définie. On va ainsi définir une fonction `du(t)` adaptée à cette syntaxe. 


In [60]:
slope=-1 # type d'échelon +1/-1
Emax=5 # amplitude (en Volt)
u0=Emax*(1-slope)/2 # tension initiale aux bornes du condensateur, en Volt
    

def E(t):
    if slope==1:
        if t<0 :
            return 0.0
        else :
            return Emax
    elif slope==-1:
        if t>0 :
            return 0.0
        else :
            return Emax
    else:
        print("Mauvaise valeur du paramètre 'slope'")
        pass

def dUc(Uc, t):
    return (E(t)-Uc)/tau

### Intégraton de l'équation différentielle

In [69]:
tau=R*C
t=np.linspace(-tau,10*tau,400) # vecteur de temps
Uc = itg.odeint(dUc, [u0], t)
Eref=np.array([E(x) for x in t])

### Représentation graphique

In [ ]:
fig=plt.figure(1, figsize=(15, 10))
plt.rcParams.update({'font.size': 22}) # change font size
plt.plot(t/tau,Eref/Emax,"g-", linewidth=4)
plt.plot(t/tau,Uc/Emax,"b-", linewidth=4)
plt.legend([r'$E(t)$',r'$u(t)$'])
plt.xlabel(r'$t/\tau$')
plt.ylabel(r'$u(t)/E_{max}$ ')
plt.grid()
plt.plot([ -1, 10 ], [(1+slope)/2, (1+slope)/2], "y--", linewidth=2)
plt.plot([ 1, 1 ], [-.1, 1.10 ], "y--", linewidth=2)
plt.plot([ 3, 3 ], [-.1, 1.10 ], "y--", linewidth=2)
plt.plot([ 0, 1 ], [ (1-slope)/2, (1+slope)/2 ], "r-.", linewidth=2)
plt.plot([1],[(1+slope)/2],'o',color='red',markersize=10)
plt.plot([1],[(1-slope)/2+slope*.63],'o',color='black',markersize=8)
plt.plot([3],[(1-slope)/2+slope*.95],'o',color='black',markersize=8)
plt.annotate("63\%", (1, (1-slope)/2+slope*0.63), (1.5, (1-slope)/2+slope*0.52), arrowprops={"arrowstyle":"->"})
plt.annotate("95\%", (3, (1-slope)/2+slope*0.95), (3.5, (1-slope)/2+slope*0.84), arrowprops={"arrowstyle":"->"})
plt.xlim(-1,7)
plt.ylim(-0.1,1.1)
plt.tight_layout()
plt.show()